In [2]:
import pandas as pd

df = pd.read_csv("/Users/S340/Documents/Octavo/Analítica de datos /airbnb-proyecto2-20252/airbnb_limpio.csv")

print("Dimensiones del dataset:", df.shape)
df.head()




Dimensiones del dataset: (5562, 80)


,id,source,name,host_id,host_name,host_since,host_is_superhost,host_listings_count,host_total_listings_count,host_verifications,...,has_free_street_parking,has_private_entrance,has_essentials,has_heating,has_wifi,has_pets_allowed,has_hot_water,has_self_check_in,has_freezer,has_exercise_equipment
0,18744501,city scrape,"""Artist´s Creative Residence"" 100m² im Zentrum",129635321,Sylvia,2017-05-09,f,1.0,1.0,"['email', 'phone']",...,1,1,1,1,1,0,1,0,0,0
1,23356842,city scrape,"""Bohemian Residency"" (Central & Quiet) * * * * *",150173398,Vincent,2017-09-11,t,2.0,3.0,"['email', 'phone']",...,1,1,1,1,1,0,1,1,0,0
2,819658084391291386,city scrape,"""Feel at Home"" Flat at the Lerchenauer See",29225873,Skandar,2015-03-12,f,1.0,1.0,"['email', 'phone']",...,0,0,1,1,1,1,1,0,0,1
3,34677963,city scrape,"""Little Star"" Schlafoase im Zentrum",28482431,Adriana,2015-02-28,f,5.0,5.0,"['email', 'phone']",...,0,1,1,1,1,0,1,1,0,0
4,34431776,city scrape,"""Moonlight"" Schlafoase mitten im Szenenviertel",28482431,Adriana,2015-02-28,f,5.0,5.0,"['email', 'phone']",...,0,1,1,1,1,0,1,1,0,0


In [3]:
df["recommended"] = df["host_is_superhost"].map({"t": 1, "f": 0})

df["recommended"].value_counts()

recommended
0    4329
1    1233
Name: count, dtype: int64

In [4]:
numeric_df = df.select_dtypes(include=["int64", "float64"])

# Incluir también la variable objetivo
numeric_df["recommended"] = df["recommended"]

numeric_df.head()

,id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,...,has_private_entrance,has_essentials,has_heating,has_wifi,has_pets_allowed,has_hot_water,has_self_check_in,has_freezer,has_exercise_equipment,recommended
0,18744501,129635321,1.0,1.0,48.159077,11.569463,5,1.0,2.0,3.0,...,1,1,1,1,0,1,0,0,0,0
1,23356842,150173398,2.0,3.0,48.146576,11.603159,8,2.0,3.0,6.0,...,1,1,1,1,0,1,1,0,0,1
2,819658084391291386,29225873,1.0,1.0,48.196742,11.533000,3,1.0,1.0,1.0,...,0,1,1,1,1,1,0,0,1,0
3,34677963,28482431,5.0,5.0,48.130140,11.578380,2,1.0,1.0,1.0,...,1,1,1,1,0,1,1,0,0,0
4,34431776,28482431,5.0,5.0,48.130810,11.578320,2,1.0,1.0,1.0,...,1,1,1,1,0,1,1,0,0,0


In [5]:
X = numeric_df.drop(columns=["recommended"])
y = numeric_df["recommended"]

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train.shape, X_test.shape

((4449, 63), (1113, 63))

In [7]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Creamos el modelo (con más iteraciones para asegurar convergencia)
log_reg = LogisticRegression(
    max_iter=1000,
    n_jobs=-1,       # usa todos los núcleos disponibles
    solver="lbfgs"
)

# Entrenar
log_reg.fit(X_train_scaled, y_train)

# Predicciones en test
y_pred = log_reg.predict(X_test_scaled)
y_proba = log_reg.predict_proba(X_test_scaled)[:, 1]

# Métricas básicas
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy en test: {acc:.4f}\n")

print("Matriz de confusión:")
print(confusion_matrix(y_test, y_pred))

print("\nReporte de clasificación:")
print(classification_report(y_test, y_pred))

Accuracy en test: 0.8149

Matriz de confusión:
[[826  40]
 [166  81]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.83      0.95      0.89       866
           1       0.67      0.33      0.44       247

    accuracy                           0.81      1113
   macro avg       0.75      0.64      0.66      1113
weighted avg       0.80      0.81      0.79      1113



In [9]:
import joblib
import os

# Crear el directorio si no existe
os.makedirs("../modelos", exist_ok=True)

joblib.dump(log_reg, "../modelos/modelo_clasificacion_lr.pkl")
joblib.dump(scaler, "../modelos/escalador_clasificacion.pkl")

['../modelos/escalador_clasificacion.pkl']